# AutoGluon 滾動訓練（三年訓練、預測第四年）— 依序版

- **輸入**：`data/autogluon_ready/{0900,0915,0930}/merged_for_autogluon_*.csv`（由 `merge_output2_for_autogluon.py` 產出）。
- **流程**：依序跑 0900 → 0915 → 0930 三組；每組用**前三年**資料訓練、預測**第四年**；輸出至 `data/models/{cutoff}/`。
- **斷線續跑**：若輸出資料夾已有 `predictions.csv` 則跳過該次訓練，直接載入既有結果。
- **並行版**：若要同時跑三個截點互不干擾，請使用 `train_autogluon_colab_0900.ipynb`、`train_autogluon_colab_0915.ipynb`、`train_autogluon_colab_0930.ipynb`（各開一個 Colab）。
- **訓練年比較**：若要迴圈 train_years=2,3,4,5 比較哪個較佳，請使用上述三個單一截點 notebook（已內建 TRAIN_YEARS_LIST）。
- **Colab**：將 merge 後的資料上傳至 Drive 專案 `data/autogluon_ready/`，下方設定 `DRIVE_PROJECT_ROOT`。

## 1. 掛載 Google Drive（Colab 必跑；本機可略）

In [ ]:
try:
    from google.colab import drive
    drive.mount("/content/drive")
    IN_COLAB = True
except Exception:
    IN_COLAB = False
print("Colab:", IN_COLAB)

Mounted at /content/drive
Colab: True


## 2. 路徑與參數

In [ ]:
from pathlib import Path

# ========== 路徑設定（Colab 請先跑上方「掛載 Drive」） ==========
DRIVE_PROJECT_ROOT = "/content/drive/MyDrive/Thesis-AutoGluon-TXF-Research"
LOCAL_PROJECT_ROOT = "/Volumes/Transcend/thesis/github_clone/Thesis-AutoGluon-TXF-Research"

PROJECT_ROOT = Path(DRIVE_PROJECT_ROOT) if IN_COLAB else Path(LOCAL_PROJECT_ROOT)
DATA_ROOT = PROJECT_ROOT / "data"

# 依序跑 0900 → 0915 → 0930 三組
CUTOFFS = ("0900", "0915", "0930")
TRAIN_YEARS = 3  # 三年訓練、預測第四年
LABEL = "target_return"
TIME_LIMIT = 30  # 秒；每段訓練時間

print("PROJECT_ROOT:", PROJECT_ROOT)
print("CUTOFFS:", CUTOFFS, "| TRAIN_YEARS:", TRAIN_YEARS)
for c in CUTOFFS:
    p = DATA_ROOT / "autogluon_ready" / c / f"merged_for_autogluon_{c}.csv"
    print(f"  {c}: {'✅' if p.exists() else '❌'} {p}")

MERGED_CSV_PATH: /content/drive/MyDrive/2026/論文/Thesis-AutoGluon-TXF-Research/data/merged_for_autogluon_0900/merged_for_autogluon_0900.csv
ROLL_OUTPUT: /Volumes/Transcend/thesis/github_clone/Thesis-AutoGluon-TXF-Research/data/models
TRAIN_YEARS: 2


In [ ]:
# 路徑與參數已在上方 cell 設定；此 cell 可略過或刪除
# 若需改截點，請修改上方 CUTOFF = "0900" | "0915" | "0930"

NameError: name 'IN_COLAB' is not defined

## 3. 安裝 AutoGluon（Colab 通常需執行一次）

In [ ]:
!pip3 install autogluon.tabular

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.9/98.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.2 MB/s eta 0:00:00


## 4. 載入資料（已移至下方訓練 cell，依 cutoff 分別載入）


In [ ]:
# 資料載入已移至下方「滾動訓練」cell，依每個 cutoff 分別載入


Shape: (2271, 52)
Years in data: [np.int32(2013), np.int32(2014), np.int32(2015), np.int32(2016), np.int32(2017), np.int32(2018), np.int32(2019), np.int32(2020), np.int32(2021), np.int32(2022), np.int32(2023)]
Predict years (train 2yr → predict 1yr): [np.int32(2015), np.int32(2016), np.int32(2017), np.int32(2018), np.int32(2019), np.int32(2020), np.int32(2021), np.int32(2022), np.int32(2023)]


In [ ]:
# Sharpe 回測（與 working/1129 intraday_macro_features_test sharpe.ipynb 相同邏輯）
COST_PER_TRADE = 0.0005
TRADE_THRESHOLD = 0.0001

def compute_sharpe_backtest(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    positions = np.zeros_like(y_pred, dtype=float)
    positions[y_pred > TRADE_THRESHOLD] = 1.0
    positions[y_pred < -TRADE_THRESHOLD] = -1.0
    strategy_returns = positions * y_true
    trades = np.sum(np.abs(np.diff(np.insert(positions, 0, 0))) > 0)
    total_cost = trades * COST_PER_TRADE
    mean_return_with_cost = (np.sum(strategy_returns) - total_cost) / max(len(strategy_returns), 1)
    std_return = np.std(strategy_returns)
    if std_return == 0:
        return 0.0
    return float(mean_return_with_cost / (std_return + 1e-9) * np.sqrt(252))

## 5. 滾動訓練：依序 0900→0915→0930，三年訓練預測第四年，已存在則跳過


In [14]:
import torch
import gc
import shutil
import json
import numpy as np
import pandas as pd
from datetime import datetime
from autogluon.tabular import TabularPredictor
from pathlib import Path

DATA_ROOT = Path(DRIVE_PROJECT_ROOT) / "data" if IN_COLAB else Path(LOCAL_PROJECT_ROOT) / "data"

HAS_GPU = torch.cuda.is_available()
print(f"系統檢查: GPU {'可用 ✅' if HAS_GPU else '未偵測到 ⚠️ (將使用 CPU)'}")
print(f"訓練設定: 依序 {CUTOFFS}, 滾動視窗={TRAIN_YEARS}年→預測第四年, 限時={TIME_LIMIT}秒")
print(f"斷線續跑: 若 roll_YYYY 內已有 predictions.csv 則跳過\n")

all_summary = []

for cutoff in CUTOFFS:
    merged_path = DATA_ROOT / "autogluon_ready" / cutoff / f"merged_for_autogluon_{cutoff}.csv"
    if not merged_path.exists():
        print(f"⏭️ 跳過 {cutoff}: 檔案不存在 {merged_path}")
        continue

    print(f"\n{'#'*50}")
    print(f"# 截點 {cutoff}")
    print(f"{'#'*50}")

    df = pd.read_csv(merged_path)
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.dropna(subset=["date"]).drop(columns=["datetime"], errors="ignore")
    df["year"] = df["date"].dt.year
    df = df.dropna()
    if LABEL not in df.columns:
        print(f"❌ {cutoff}: 無 {LABEL} 欄位")
        continue

    years = sorted(df["year"].unique())
    predict_years = [y for y in years if all((y - i) in years for i in range(1, TRAIN_YEARS + 1))]
    predict_years = sorted(set(predict_years))
    print(f"  資料: {len(df)} 列, 預測年: {predict_years}")

    ROLL_OUTPUT = DATA_ROOT / "models" / cutoff
    ROLL_OUTPUT.mkdir(parents=True, exist_ok=True)
    summary_list = []
    per_year_reports = {}

    for predict_year in predict_years:
        required_train_years = range(predict_year - TRAIN_YEARS, predict_year)
        path_roll = ROLL_OUTPUT / f"roll_{predict_year}"

        if (path_roll / "predictions.csv").exists():
            print(f"  ⏭️ {predict_year}: 已存在，跳過訓練")
            try:
                with open(path_roll / "metrics.json") as f:
                    m = json.load(f)
                summary_list.append({
                    "cutoff": cutoff, "predict_year": int(predict_year),
                    "train_period": m.get("train_period", ""),
                    "rmse": m.get("rmse"), "sharpe": m.get("sharpe"),
                    "best_model": m.get("best_model", ""), "model_path": f"roll_{predict_year}", "skipped": True
                })
                lb = pd.read_csv(path_roll / "leaderboard_with_metrics.csv") if (path_roll / "leaderboard_with_metrics.csv").exists() else pd.DataFrame()
                perf = pd.read_csv(path_roll / "models_performance.csv") if (path_roll / "models_performance.csv").exists() else pd.DataFrame()
                fi = pd.read_csv(path_roll / "feature_importance_all_models.csv") if (path_roll / "feature_importance_all_models.csv").exists() else pd.DataFrame()
                per_year_reports[int(predict_year)] = {"leaderboard": lb, "model_performance": perf, "feature_importance": fi}
            except Exception as e:
                print(f"    ⚠️ 載入既有結果失敗: {e}")
            continue

        missing_years = [y for y in required_train_years if y not in df["year"].unique()]
        if missing_years:
            print(f"  ❌ 跳過 {predict_year}: 缺少 {missing_years}")
            continue

        train_df = df[df["year"].isin(required_train_years)].copy()
        test_df = df[df["year"] == predict_year].copy()
        train_ag = train_df.drop(columns=["date", "datetime", "year"], errors="ignore").dropna()
        test_ag = test_df.drop(columns=["date", "datetime", "year"], errors="ignore").dropna()

        if len(train_ag) < 50 or len(test_ag) < 10:
            print(f"  ⚠️ 跳過 {predict_year}: 資料過少")
            continue

        print(f"  🚀 {predict_year}: 訓練 {min(required_train_years)}-{max(required_train_years)} → 預測 {predict_year} ({len(train_ag)}/{len(test_ag)} 筆)")

        if 'predictor' in locals():
            del predictor
        gc.collect()
        torch.cuda.empty_cache() if HAS_GPU else None

        path_roll.mkdir(parents=True, exist_ok=True)

        try:
            predictor = TabularPredictor(
                label=LABEL, problem_type="regression", eval_metric="rmse", path=str(path_roll),
            ).fit(
                train_data=train_ag, time_limit=TIME_LIMIT, presets="best_quality",
                dynamic_stacking=True, ag_args_fit={'num_gpus': 1} if HAS_GPU else {'num_gpus': 0}
            )
        except Exception as e:
            print(f"  ❌ 訓練錯誤 {predict_year}: {e}")
            continue

        preds = predictor.predict(test_ag)
        y_true = test_ag[LABEL].values
        y_pred = np.asarray(preds)
        rmse = float(np.sqrt(np.mean((y_pred - y_true) ** 2)))
        sharpe = compute_sharpe_backtest(y_true, y_pred)

        lb = predictor.leaderboard(test_ag, silent=True)
        model_names = lb["model"].tolist()
        model_perf_rows = []
        fi_rows = []
        for model_name in model_names:
            try:
                preds_m = np.asarray(predictor.predict(test_ag, model=model_name))
                model_perf_rows.append({
                    "predict_year": int(predict_year), "train_period": f"{min(required_train_years)}-{max(required_train_years)}",
                    "model": model_name, "rmse": float(np.sqrt(np.mean((preds_m - y_true) ** 2))),
                    "sharpe": compute_sharpe_backtest(y_true, preds_m),
                })
            except Exception:
                pass
            try:
                fi_m = predictor.feature_importance(data=test_ag, model=model_name).reset_index().rename(columns={"index": "feature"})
                fi_m["model"] = model_name
                fi_m["predict_year"] = int(predict_year)
                fi_rows.append(fi_m)
            except Exception:
                pass

        df_model_perf = pd.DataFrame(model_perf_rows)
        lb_with_metrics = lb.merge(df_model_perf, on="model", how="left")
        df_fi_all = pd.concat(fi_rows, ignore_index=True) if fi_rows else pd.DataFrame()

        lb.to_csv(path_roll / "leaderboard.csv", index=False)
        lb_with_metrics.to_csv(path_roll / "leaderboard_with_metrics.csv", index=False)
        df_model_perf.to_csv(path_roll / "models_performance.csv", index=False)
        df_fi_all.to_csv(path_roll / "feature_importance_all_models.csv", index=False)

        out_pred = test_df.loc[test_ag.index].copy()
        out_pred["pred"] = y_pred
        out_pred[["date", LABEL, "pred"]].to_csv(path_roll / "predictions.csv", index=False)

        metrics = {
            "cutoff": cutoff, "predict_year": int(predict_year),
            "train_period": f"{min(required_train_years)}-{max(required_train_years)}",
            "rmse": rmse, "sharpe": sharpe, "best_model": predictor.model_best,
            "num_models": len(model_names), "model_path": f"roll_{predict_year}",
        }
        with open(path_roll / "metrics.json", "w", encoding="utf-8") as f:
            json.dump(metrics, f, indent=2, ensure_ascii=False)

        summary_list.append({
            "cutoff": cutoff, "predict_year": int(predict_year),
            "train_period": metrics["train_period"], "rmse": rmse, "sharpe": sharpe,
            "best_model": predictor.model_best, "num_models": len(model_names),
            "model_path": f"roll_{predict_year}", "skipped": False
        })
        per_year_reports[int(predict_year)] = {"leaderboard": lb_with_metrics, "model_performance": df_model_perf, "feature_importance": df_fi_all}
        print(f"  🎉 完成 {predict_year}: Sharpe={sharpe:.4f}, RMSE={rmse:.5f}, Best={predictor.model_best}")

    if summary_list:
        df_summary = pd.DataFrame(summary_list)
        summary_path = ROLL_OUTPUT / "rolling_summary_final.csv"
        df_summary.to_csv(summary_path, index=False)
        print(f"\n✅ {cutoff} 完成，總表: {summary_path}")
        all_summary.append(df_summary)

        excel_path = ROLL_OUTPUT / "rolling_models_by_year.xlsx"
        try:
            with pd.ExcelWriter(excel_path, engine="openpyxl") as writer:
                for year in sorted(per_year_reports.keys()):
                    report = per_year_reports[year]
                    sheet_name = str(year)[:31]
                    startrow = 0
                    if not report["leaderboard"].empty:
                        pd.DataFrame({"section": ["leaderboard_with_metrics"]}).to_excel(writer, sheet_name=sheet_name, index=False, header=False, startrow=startrow)
                        startrow += 1
                        report["leaderboard"].to_excel(writer, sheet_name=sheet_name, index=False, startrow=startrow)
                        startrow += len(report["leaderboard"]) + 2
                    if not report["model_performance"].empty:
                        pd.DataFrame({"section": ["model_performance"]}).to_excel(writer, sheet_name=sheet_name, index=False, header=False, startrow=startrow)
                        startrow += 1
                        report["model_performance"].to_excel(writer, sheet_name=sheet_name, index=False, startrow=startrow)
                        startrow += len(report["model_performance"]) + 2
                    if not report["feature_importance"].empty:
                        pd.DataFrame({"section": ["feature_importance_all_models"]}).to_excel(writer, sheet_name=sheet_name, index=False, header=False, startrow=startrow)
                        startrow += 1
                        report["feature_importance"].to_excel(writer, sheet_name=sheet_name, index=False, startrow=startrow)
            print(f"  Excel: {excel_path}")
        except Exception as e:
            print(f"  ⚠️ Excel 輸出失敗: {e}")

if all_summary:
    combined = pd.concat(all_summary, ignore_index=True)
    combined.to_csv(DATA_ROOT / "models" / "rolling_summary_all_cutoffs.csv", index=False)
    print(f"\n✅ 全部完成！彙總: data/models/rolling_summary_all_cutoffs.csv")
    display(combined)
else:
    print("\n❌ 沒有產生任何結果，請檢查資料或路徑。")



系統檢查: GPU 未偵測到 ⚠️ (將使用 CPU)
訓練設定: 滾動視窗=2年, 預測=[np.int32(2015), np.int32(2016), np.int32(2017), np.int32(2018), np.int32(2019), np.int32(2020), np.int32(2021), np.int32(2022), np.int32(2023)], 限時=30秒

🚀 開始處理年份: 2015
📊 數據統計: 訓練集 435 筆 (2013-2014), 測試集 202 筆 (2015)


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Pytorch Version:    2.9.0+cpu
CUDA Version:       CUDA is not available
Memory Avail:       10.42 GB / 12.67 GB (82.3%)
Disk Space Avail:   81.73 GB / 107.72 GB (75.9%)
Presets specified: ['best_quality']
Using hyperparameters preset: hyperparameters='zeroshot'
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked overfitting.
	Runnin

✅ 特徵重要性已輸出
🎉 完成 2015: Sharpe=2.1658, RMSE=0.00795, Best=WeightedEnsemble_L2

🚀 開始處理年份: 2016
📊 數據統計: 訓練集 421 筆 (2014-2015), 測試集 194 筆 (2016)


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Pytorch Version:    2.9.0+cpu
CUDA Version:       CUDA is not available
Memory Avail:       10.38 GB / 12.67 GB (81.9%)
Disk Space Avail:   81.72 GB / 107.72 GB (75.9%)
Presets specified: ['best_quality']
Using hyperparameters preset: hyperparameters='zeroshot'
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked overfitting.
	Runnin

✅ 特徵重要性已輸出
🎉 完成 2016: Sharpe=-2.8726, RMSE=0.00811, Best=WeightedEnsemble_L2

🚀 開始處理年份: 2017
📊 數據統計: 訓練集 396 筆 (2015-2016), 測試集 170 筆 (2017)


...

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Pytorch Version:    2.9.0+cpu
CUDA Version:       CUDA is not available
Memory Avail:       10.45 GB / 12.67 GB (82.5%)
Disk Space Avail:   81.61 GB / 107.72 GB (75.8%)
Presets specified: ['best_quality']
Using hyperparameters preset: hyperparameters='zeroshot'
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked overfitting.
	Runnin

✅ 特徵重要性已輸出
🎉 完成 2022: Sharpe=-0.4351, RMSE=0.00783, Best=WeightedEnsemble_L2

🚀 開始處理年份: 2023
📊 數據統計: 訓練集 428 筆 (2021-2022), 測試集 212 筆 (2023)


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Pytorch Version:    2.9.0+cpu
CUDA Version:       CUDA is not available
Memory Avail:       10.45 GB / 12.67 GB (82.4%)
Disk Space Avail:   81.59 GB / 107.72 GB (75.7%)
Presets specified: ['best_quality']
Using hyperparameters preset: hyperparameters='zeroshot'
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked overfitting.
	Runnin

✅ 特徵重要性已輸出
🎉 完成 2023: Sharpe=-1.8960, RMSE=0.00518, Best=WeightedEnsemble_L2

✅ 所有年份滾動訓練完成！總表已存至: /content/drive/MyDrive/Thesis-AutoGluon-TXF-Research/data/models/rolling_summary_final.csv


,predict_year,train_period,rmse,sharpe,best_model,model_path
0,2015,2013-2014,0.007954,2.165834,WeightedEnsemble_L2,roll_2015_20260208_161232
1,2016,2014-2015,0.008113,-2.872606,WeightedEnsemble_L2,roll_2016_20260208_161308
2,2017,2015-2016,0.004380,-5.815242,WeightedEnsemble_L3,roll_2017_20260208_161344
3,2018,2016-2017,0.006476,-4.696000,WeightedEnsemble_L2,roll_2018_20260208_161427
4,2019,2017-2018,0.004631,-3.015459,WeightedEnsemble_L3,roll_2019_20260208_161502
5,2020,2018-2019,0.010140,-0.419958,WeightedEnsemble_L2,roll_2020_20260208_161545
6,2021,2019-2020,0.008684,-0.707520,WeightedEnsemble_L2,roll_2021_20260208_161624
7,2022,2020-2021,0.007833,-0.435050,WeightedEnsemble_L2,roll_2022_20260208_161705
8,2023,2021-2022,0.005180,-1.895955,WeightedEnsemble_L2,roll_2023_20260208_161748


## 6. 彙總表（數值輸出供本地分析）

In [12]:
# 輸出：data/models/{0900,0915,0930}/roll_YYYY/ 含 predictions.csv、metrics.json、leaderboard 等
# 彙總表見上方訓練 cell 輸出，或 data/models/rolling_summary_all_cutoffs.csv
summary_path = Path(DRIVE_PROJECT_ROOT if IN_COLAB else LOCAL_PROJECT_ROOT) / "data" / "models" / "rolling_summary_all_cutoffs.csv"
if summary_path.exists():
    display(pd.read_csv(summary_path))
else:
    print("尚未產生彙總表，請先執行上方滾動訓練 cell。")

No rolls completed. Check predict_years and data.
